# Zwischenankunktszeiten

In [1]:
from configs.config import get_path
import src.utils.converter as convert

import json
import numpy as np
import pandas as pd

In [2]:
basic_data_path = get_path("data", "basic")

In [3]:
# Datei laden
file_path = basic_data_path / "jobshop_instances.json"

with open(file_path, "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]
df_instance = convert.jssp_dict_to_df(instance)
df_instance

,Job,Operation,Machine,Processing Time
0,job 0,0,M0,29
1,job 0,1,M1,78
2,job 0,2,M2,9
3,job 0,3,M3,36
4,job 0,4,M4,49
...,...,...,...,...
95,job 9,5,M9,76
96,job 9,6,M5,47
97,job 9,7,M3,52
98,job 9,8,M4,90


## I) Mittlere Zwischenankunftszeit t_a

### 1. Vektor der Bearbeitungszeiten auf der Engpassmaschine

#### a) Bestimmung der Engpassmaschine (7.12)

In [4]:
def get_engpassmaschine(df, debug=False):
    """
    Ermittelt die Maschine mit der höchsten Gesamtbearbeitungszeit (Bottleneck) aus einem DataFrame.

    Parameter:
    - df: DataFrame mit Spalten ['Job','Machine','Processing Time']
          Machine kann entweder als int oder als String 'M{int}' vorliegen.
    - debug: Wenn True, wird die vollständige Auswertung der Maschinenbelastung ausgegeben.

    Rückgabe:
    - Index der Engpassmaschine (int)
    """
    d = df.copy()
    # Falls Machine als 'M0','M1',... vorliegt, entfernen wir das 'M'
    if d['Machine'].dtype == object:
        d['Machine'] = d['Machine'].str.lstrip('M').astype(int)
    # Gesamtbearbeitungszeit pro Maschine
    usage = d.groupby('Machine')['Processing Time'].sum().to_dict()
    if debug:
        print("Maschinenbelastung (Gesamtverarbeitungszeit):")
        for m, total in sorted(usage.items()):
            print(f"  M{m}: {total}")
    # Maschine mit maximaler Gesamtzeit
    return max(usage, key=usage.get)

In [5]:
engpassmaschine = get_engpassmaschine(df_instance, debug = False)
engpassmaschine

3

#### b) Vektor der Bearbeitungszeiten auf der Engpassmaschine erstellen

In [6]:
def get_vec_t_b_mmax(df):
    """
    Ermittelt für jeden Job die Bearbeitungszeit auf der Engpassmaschine.

    Parameter:
    - df: DataFrame mit Spalten ['Job','Machine','Processing Time']

    Rückgabe:
    - Liste der Bearbeitungszeiten auf der Engpassmaschine, in der Reihenfolge
      der ersten Vorkommen der Jobs in df['Job'].
    """
    # 1) Kopie und Machine-Spalte in int umwandeln, falls nötig
    d = df.copy()
    if d['Machine'].dtype == object:
        d['Machine'] = d['Machine'].str.lstrip('M').astype(int)

    # 2) Engpassmaschine bestimmen
    eng = get_engpassmaschine(d)

    # 3) Job-Reihenfolge festlegen
    job_order = d['Job'].unique().tolist()

    # 4) Zeiten auf Engpassmaschine extrahieren
    proc_on_eng = d[d['Machine'] == eng].set_index('Job')['Processing Time'].to_dict()

    # 5) Vektor aufbauen (0, wenn ein Job die Maschine nicht nutzt)
    vec = [proc_on_eng.get(job, 0) for job in job_order]
    return vec

In [7]:
vec_t_b_mmax = get_vec_t_b_mmax(df_instance)
vec_t_b_mmax

[36, 69, 39, 98, 26, 95, 61, 79, 76, 52]

### 2. Job-Wahrscheinlichkeiten

In [8]:
# Anzahl unterschiedlicher Jobs
n_jobs = df_instance['Job'].nunique()

p = [1.0 / n_jobs] * n_jobs
p

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

### 3. Berechne die mittlere Zwischenankunftszeit t_a nach Formel (7.16) 
> All-in-One

In [9]:
def calculate_mean_interarrival_time(df, u_b_mmax: float = 0.9) -> float:
    """
    Berechnet die mittlere Interarrival-Zeit t_a für ein DataFrame,
    sodass die Engpassmaschine mit Auslastung u_b_mmax (< 1.0) betrieben wird.

    Parameter:
    - df: DataFrame mit Spalten ['Job','Machine','Processing Time']
    - u_b_mmax: Ziel-Auslastung der Engpassmaschine (z.B. 0.9)

    Rückgabe:
    - t_a: mittlere Interarrival-Zeit, gerundet auf 2 Dezimalstellen
    """
    # Anzahl der unterschiedlichen Jobs
    n_jobs = df['Job'].nunique()
    # Gleichverteilung über die Jobs
    p = [1.0 / n_jobs] * n_jobs

    # Vektor der Bearbeitungszeiten auf der Engpassmaschine
    vec_t_b_mmax = get_vec_t_b_mmax(df)

    # Berechnung der mittleren Interarrival-Zeit
    t_a = sum(p[i] * vec_t_b_mmax[i] for i in range(n_jobs)) / u_b_mmax
    return round(t_a, 2)

In [10]:
t_a =  calculate_mean_interarrival_time(df_instance, u_b_mmax = 0.9)
t_a

70.11

## II) Ankunftszeiten

#### Generierung exponentiell verteilte Zwischenankunftszeiten mit Mittelwert t_a 
#### mit eine zufällige Job-Reihenfolge

In [11]:
def generate_arrival_dataframe(df_jssp: pd.DataFrame,
                               t_a: float,
                               perm_seed: int = 12,
                               exp_seed: int = 123) -> pd.DataFrame:
    """
    Erzeugt ein DataFrame mit zufälligen Ankunftszeiten für jeden Job.

    Parameter:
    - df_jssp: DataFrame mit mindestens der Spalte 'Job'.
    - t_a: mittlere Interarrival-Zeit (Skala für Exponentialverteilung).
    - perm_seed: Zufallssamen für die Permutation der Jobs.
    - exp_seed: Zufallssamen für die Exponentialverteilung.

    Rückgabe:
    - df_arrivals: DataFrame mit Spalten ['Job','Arrival'], sortiert nach 'Arrival'.
    """
    # 1) Eindeutige Job-Namen und Anzahl
    job_names = df_jssp['Job'].unique().tolist()
    n_jobs = len(job_names)

    # 2) Permutation der Job-Reihenfolge
    np.random.seed(perm_seed)
    shuffled_jobs = list(np.random.permutation(job_names))

    # 3) Exponentiell verteilte Interarrival-Zeiten
    np.random.seed(exp_seed)
    interarrival_times = np.random.exponential(scale=t_a, size=n_jobs)

    # 4) Absolute Ankunftszeiten (kumuliert und gerundet)
    arrival_times = np.cumsum(interarrival_times)
    arrival_times = np.round(arrival_times, 2)

    # 5) Kombination und Sortierung
    df_arrivals = pd.DataFrame({
        'Job': shuffled_jobs,
        'Arrival': arrival_times
    }).sort_values('Arrival').reset_index(drop=True)

    return df_arrivals

In [12]:
df_arrivals = generate_arrival_dataframe(df_instance, t_a)
df_arrivals

,Job,Arrival
0,job 5,83.59
1,job 8,107.22
2,job 7,125.26
3,job 0,181.45
4,job 4,270.56
5,job 9,309.13
6,job 3,586.13
7,job 2,667.09
8,job 1,713.06
9,job 6,747.96


### Export

In [13]:
df_instance.to_csv(basic_data_path / "00_instance.csv", index = False)
df_arrivals.to_csv(basic_data_path / "00_arrivals.csv", index = False)